In [91]:
import pandas as pd
import numpy as np

lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_17916\3073547063.py:4: DtypeWarning: Columns (1,7,8,9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")


In [92]:
# -----------------------------------------
# ----- Step 1: Results with reportable limits -> flag implausible results

lab_clean.sample()

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
3650753,12398,52305589204,2019,2019-05-23,H10272,ERITRÒCITS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,"4,94",M/mcL,"4,5","5,5",4.94,M/mcL,NaN,NaN,n1


In [21]:
lab_clean.dtypes

codi_p               int64
peticio_id          object
any                  int64
data                object
codi_prova          object
prova               object
resultat            object
unitat_mesura       object
ref_min             object
ref_max             object
clean_result        object
clean_unit          object
comentari           object
comentari_unitat    object
num_type            object
dtype: object

In [79]:
lab_clean[lab_clean['peticio_id'] == "40905666215"]

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
1024760,5909,40905666215,2010,2010-04-09,Q90361,FILTRAT GLOMERULAR (ESTIMACIÓ SEGONS MDRD4),>60,mL/min,NaN,NaN,>60,mL/min,NaN,done,n2
1855607,5909,40905666215,2010,2010-04-09,Q52485,TIROXINA (LLIURE); C.SUBST. (sèrum),"2,1",ng/dL,"0,7","1,8",2.1,ng/dL,NaN,done,n1
2135875,5909,40905666215,2010,2010-04-09,Q52385,TIROTROPINA (SÈRUM); C.SUBST.ARB.,"1,92",mUI/L,"0,27","4,2",1.92,m[IU]/L,NaN,done,n1
2459197,5909,40905666215,2010,2010-04-09,Q02458,ALBÚMINA; C.MASSA (orina),36,mg/L,0,20,36,mg/L,NaN,done,n1
2776490,5909,40905666215,2010,2010-04-09,Q35485,IÓ POTASSI (SÈRUM); C.SUBST. «PART DE CONJUNT»,"4,1",mEqu/L,"3,5","5,5",4.1,meq/L,NaN,done,n1
5729945,5909,40905666215,2010,2010-04-09,Q12785,COLESTEROL D'HDL; C. SUBST. (sèrum),80,mg/dL,45,90,80,mg/dL,NaN,done,n1
5866017,5909,40905666215,2010,2010-04-09,Q53585,TRIGLICÈRID; C.SUBST. (sèrum),109,mg/dL,50,200,109,mg/dL,NaN,done,n1
6146179,5909,40905666215,2010,2010-04-09,Q32685,GLUCOSA (SÈRUM); C.SUBST.,86,mg/dL,55,115,86,mg/dL,NaN,done,n1
6153652,5909,40905666215,2010,2010-04-09,Q15185,CREATININI (SÈRUM); C.SUBST,",86",mg/dL,"0,5","0,9",0.86,mg/dL,NaN,done,n1
7404880,5909,40905666215,2010,2010-04-09,Q13285,COLESTEROL; C.SUBST. (sèrum),209,mg/dL,180,250,209,mg/dL,NaN,done,n1


In [ ]:
# Logic rules
# From: Zhu J, Wang H, Wang B, Hao X, Cui W, Duan Y, et al. Combined strategy of knowledge-based rule selection and historical data percentile‐based range determination to improve an autoverification system for clinical chemistry test results. J Clin Lab Anal. 2022;36(2):e24233.

# TBIL -DBIL > 0
# TP - ALB > 0
# TC - HDL -LDL > 0
# NA - CL - TCO2 - AG = 0





In [93]:
lab_clean[(lab_clean['comentari'] == "literal")]

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
43214,3611,61308048295,2018,2018-06-13,H10940,NEUTRÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,NoCalc,K/mcL,NaN,NaN,nc,10*3/uL,literal,done,NaN
125741,3222,70608055190,2018,2018-07-06,H10940,NEUTRÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,NoCalc,K/mcL,NaN,NaN,nc,10*3/uL,literal,done,NaN
138909,9860,100408077668,2017,2017-10-04,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,NoCalc,K/mcL,NaN,NaN,nc,10*3/uL,literal,done,NaN
139151,12515,111508085399,2016,2016-11-15,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,NoCalc,K/mcL,NaN,NaN,nc,10*3/uL,literal,done,NaN
139323,1975,70608053358,2017,2017-07-06,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,NoCalc,K/mcL,NaN,NaN,nc,10*3/uL,literal,done,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7814119,83,72908059905,2020,2020-07-29,Q32685,GLUCOSA (SÈRUM); C.SUBST.,MHE,mg/dL,70,115,MHE,mg/dL,literal,done,NaN
7815386,2923,80708059148,2016,2016-08-07,H11840,BASÒFILS (HEMOGRAMA); FR.NOM. «PART DE CONJUNT»,--,%,0,2,nc,%,literal,done,NaN
7815574,13454,111408092347,2018,2018-11-14,Q32685,GLUCOSA (SÈRUM); C.SUBST.,NM,mg/dL,70,115,NM,mg/dL,literal,done,NaN
7819112,4479,12208006286,2019,2019-01-22,Q32685,GLUCOSA (SÈRUM); C.SUBST.,MHE,mg/dL,70,115,MHE,mg/dL,literal,done,NaN
